In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity




In [9]:
df=pd.read_csv('spotify_millsongdata.csv')

In [33]:
df=df.sample(10000)

In [34]:
ff=df['link']

KeyError: 'link'

In [ ]:
df.groupby('artist').count()

,song,link,text
artist,,,
'n Sync,93,93,93
ABBA,113,113,113
Ace Of Base,74,74,74
Adam Sandler,70,70,70
Adele,54,54,54
...,...,...,...
Zoegirl,38,38,38
Zornik,12,12,12
Zox,21,21,21


NOW that we have assigned the link to another column we can drop them


In [ ]:
df.drop('link', axis=1, inplace=True)


In [46]:
df.shape

(10000, 3)

In [48]:
artist_list=df.groupby('artist').count()

Now we are having 643 artist list in our dataset


In [49]:
artist_list.count()

song    625
text    625
dtype: int64

In [50]:
df['text']

44152    they called us walking corpses  \r\nunholy liv...
27445    oh girl  \r\ni looked you over once or twice  ...
52945    she gonna love him, stoned out of mind all the...
39840    well i drank till i stumbled,  \r\ni drank til...
23010    i met a babe in a backseat drive-in  \r\nback ...
                               ...                        
47459    i walk up to my door and hate to turn the key ...
37522    got the blues from my baby  \r\nleft me by the...
56316    i wake up on broken glass  \r\ni wake up on br...
52903    you see this guy  \r\nthis guy's in love with ...
36602    see the fields, ripe and white as snow  \r\nup...
Name: text, Length: 10000, dtype: object

In [52]:

p=df['text'].str.count('\n').sum()

In [53]:
df['text'] = df['text'].apply(lambda x: str(x).lower().replace(r'^\w\s',' ').replace(r'\n',' ',))


In [54]:
df.columns

Index(['artist', 'song', 'text'], dtype='object')

In [55]:

# import nltk
# it is a nlp library , it is used to process the text data
# from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

In this the similar words which are having same meaning but differ in spelling are changed into one common word


In [56]:
def token(text):
    token=nltk.word_tokenize(text)
    a=[stemmer.stem(word) for word in token]
    return " ".join(a)
    

In [58]:
# # import nltk
# # nltk.download('punkt')
# token(df['text'][0])
# token('hello world beautiful beauty')

In [59]:
df['text'].apply(lambda x:token(x))

44152    they call us walk corps unholi live dead they ...
27445    oh girl i look you over onc or twice i think t...
52945    she gon na love him , stone out of mind all th...
39840    well i drank till i stumbl , i drank till i fe...
23010    i met a babe in a backseat drive-in back in th...
                               ...                        
47459    i walk up to my door and hate to turn the key ...
37522    got the blue from my babi left me by the san f...
56316    i wake up on broken glass i wake up on broken ...
52903    you see thi guy thi guy 's in love with you re...
36602    see the field , ripe and white as snow up from...
Name: text, Length: 10000, dtype: object

In [60]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity



tdif=TfidfVectorizer(analyzer='word', stop_words='english')

In [61]:
matrix=tdif.fit_transform(df['text'])

In [62]:
# from sklearn.metrics.pairwise import cosine_similarity
# from scipy.sparse import csr_matrix

# # Assuming 'matrix' is a sparse matrix in CSR format
# sparse_matrix = csr_matrix(matrix)

# # Compute cosine similarity using sparse matrices
similar = cosine_similarity(matrix)

In [65]:
similar[0]

array([1.        , 0.        , 0.00839667, ..., 0.00783037, 0.03511215,
       0.        ])

In [70]:
df['song']

44152                         London Dungeon
27445                          Once Or Twice
52945                           Stoned Woman
39840    Being Drunk's A Lot Like Loving You
23010                            Cheese Cake
                        ...                 
47459          When Your House Is Not A Home
37522                San Francisco Bay Blues
56316                 Did You Steal My Money
52903            This Guy's In Love With You
36602                    Lord Of The Harvest
Name: song, Length: 10000, dtype: object

In [75]:
df[df['song']=='San Francisco Bay Blues'].index[0]

37522

Now we are making recommder function for getting the songs


In [ ]:
def recommder(song_name):
    